In [1]:
import pandas as pd

In [2]:
train_rides_df = pd.read_csv('DBtrainrides.csv')

In [3]:
train_rides_df.head()

,ID,line,path,eva_nr,category,station,state,city,zip,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info,arrival_delay_check,departure_delay_check
0,1573967790757085557-2407072312-14,20,Stolberg(Rheinl)Hbf Gl.44|Eschweiler-St.Jöris|...,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN,on_time,on_time
1,349781417030375472-2407080017-1,18,NaN,8000001,2,Aachen Hbf,Nordrhein-Westfalen,Aachen,52064,6.091499,50.767800,NaN,2024-07-08 00:17:00,NaN,NaN,0,0,NaN,on_time,on_time
2,7157250219775883918-2407072120-25,1,Hamm(Westf)Hbf|Kamen|Kamen-Methler|Dortmund-Ku...,8000406,4,Aachen-Rothe Erde,Nordrhein-Westfalen,Aachen,52066,6.116475,50.770202,2024-07-08 00:03:00,2024-07-08 00:04:00,2024-07-08 00:03:00,2024-07-08 00:04:00,0,0,NaN,on_time,on_time
3,349781417030375472-2407080017-2,18,Aachen Hbf,8000404,5,Aachen West,Nordrhein-Westfalen,Aachen,52072,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,NaN,NaN,0,0,NaN,on_time,on_time
4,1983158592123451570-2407080010-3,33,Herzogenrath|Kohlscheid,8000404,5,Aachen West,Nordrhein-Westfalen,Aachen,52072,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,2024-07-08 00:20:00,2024-07-08 00:21:00,0,0,NaN,on_time,on_time


In [4]:
station_df = train_rides_df[["station","category","city","zip","long","lat"]]
station_df.drop_duplicates(subset='station',inplace=True)
station_df
# Check for KeplerGl plot

C:\Users\rapha\AppData\Local\Temp\ipykernel_7064\3761056731.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  station_df.drop_duplicates(subset='station',inplace=True)


,station,category,city,zip,long,lat
0,Aachen Hbf,2,Aachen,52064,6.091499,50.767800
2,Aachen-Rothe Erde,4,Aachen,52066,6.116475,50.770202
3,Aachen West,5,Aachen,52072,6.070715,50.780360
8,Aalen Hbf,3,Aalen,73430,10.096271,48.841013
9,Achim,4,Achim,28832,9.030447,53.015990
...,...,...,...,...,...,...
34874,Potsdam Griebnitzsee,4,Potsdam,14482,13.128727,52.394432
45844,Frankfurt am Main Flughafen Fernbahnhof,3,Frankfurt am Main,60549,8.570185,50.053167
58309,Norddeich Mole,4,Norden,26506,7.158806,53.624315
305819,St. Georgen (Schwarzw),5,St. Georgen,78112,8.341955,48.123813


In [12]:
train_rides_df['partial_zipcode'] = train_rides_df['zip'].astype(str).str[:2]

grouped_df = train_rides_df.groupby('partial_zipcode').agg(
    count=('zip', 'size'),  # Count the number of entries
    entries=('partial_zipcode', lambda x: train_rides_df.loc[x.index][["station", "category", "city", "zip", "long", "lat"]].to_dict(orient='records'))
).reset_index()

grouped_df.rename(columns={'partial_zipcode': 'partial_zipcode'}, inplace=True)
grouped_df

In [21]:
station_df_delay = train_rides_df[["station", "category", "city", "zip", "long", "lat", "arrival_delay_m"]]
# Group by 'station' and calculate the mean of 'departure_delay_m', while keeping other columns
average_delay_per_station = station_df_delay.groupby(['station']).agg(
    category=('category', 'first'),           # Keep the first 'category' per station
    city=('city', 'first'),                   # Keep the first 'city' per station
    zip=('zip', 'first'),                     # Keep the first 'zip' per station
    long=('long', 'first'),                   # Keep the first 'long' per station
    lat=('lat', 'first'),                     # Keep the first 'lat' per station
    average_arrival_delay_m=('arrival_delay_m', 'mean')  # Calculate the mean of 'departure_delay_m'
).reset_index()

# Show the result
average_delay_per_station


,station,category,city,zip,long,lat,average_arrival_delay_m
0,Aachen Hbf,2,Aachen,52064,6.091499,50.767800,0.235215
1,Aachen Schanz,5,Aachen,52064,6.073840,50.769862,1.038747
2,Aachen West,5,Aachen,52072,6.070715,50.780360,0.768349
3,Aachen-Rothe Erde,4,Aachen,52066,6.116475,50.770202,1.249737
4,Aalen Hbf,3,Aalen,73430,10.096271,48.841013,0.321993
...,...,...,...,...,...,...,...
1991,Züssow,5,Züssow,17495,13.548191,53.973879,2.555288
1992,Öhringen Hbf,5,Öhringen,74613,9.502705,49.203214,1.522368
1993,Ötigheim,4,Ötigheim,76470,8.240582,48.888526,1.105769
1994,Übach-Palenberg,5,Übach-Palenberg,52531,6.097265,50.924336,1.857317


In [28]:
import plotly.graph_objects as go

# Define the color buckets for delays
color_map = {
    "0-1 min": "green",
    "1-2 min": "yellow",
    "2-3 min": "orange",
    "3-4 min": "red",
    "4-5 min": "darkred",
    "5-6 min": "purple"
}

# Create a new column for delay ranges (buckets)
def delay_bucket(delay):
    if 0 <= delay <= 1:
        return "0-1 min"
    elif 1 < delay <= 2:
        return "1-2 min"
    elif 2 < delay <= 3:
        return "2-3 min"
    elif 3 < delay <= 4:
        return "3-4 min"
    elif 4 < delay <= 5:
        return "4-5 min"
    elif 5 < delay <= 6:
        return "5-6 min"
    else:
        return "Other"

average_delay_per_station['delay_bucket'] = average_delay_per_station['average_arrival_delay_m'].apply(delay_bucket)

# Create the figure
fig = go.Figure()

# Plot each delay bucket as a separate trace
for bucket, color in color_map.items():
    # Filter the data for this bucket
    bucket_data = average_delay_per_station[average_delay_per_station['delay_bucket'] == bucket]

    # Add a scatter mapbox trace for this delay range
    fig.add_trace(go.Scattermapbox(
        lon=bucket_data['long'],
        lat=bucket_data['lat'],
        mode='markers',
        marker=dict(size=10, color=color),
        name=bucket,  # Legend entry
        hoverinfo="text",
        hovertext=bucket_data["station"] + "<br>" +
                  bucket_data["city"] + "<br>" +
                  "Avg Delay: " + bucket_data["average_arrival_delay_m"].astype(str) + " min"
    ))

# Update the map style and layout
fig.update_layout(
    mapbox_style="open-street-map",
    mapbox_zoom=4,
    mapbox_center={"lat": 51.1657, "lon": 10.4515}, # Center the map
    height=600
)

# Show the plot
fig.show()


In [7]:
train_rides_df.count()

ID                       2061357
line                     2061357
path                     1850002
eva_nr                   2061357
category                 2061357
station                  2061357
state                    2061357
city                     2061357
zip                      2061357
long                     2061357
lat                      2061357
arrival_plan             1850002
departure_plan           2061357
arrival_change           1585727
departure_change         1721431
arrival_delay_m          2061357
departure_delay_m        2061357
info                      645341
arrival_delay_check      2061357
departure_delay_check    2061357
dtype: int64

2,061,357–1,850,002 = 211,355 fewer cases in the 'path' column

In [8]:
# Step 2: Create a new DataFrame with a count of how often each 'eva_nr' is mentioned
station_mapping_df = train_rides_df.groupby(['eva_nr', 'station', 'category', 'city']).size().reset_index(name='count')

# Step 3: Optionally, reset the index for a cleaner DataFrame
station_mapping_df = station_mapping_df.reset_index(drop=True)

# Display the new DataFrame
station_mapping_df

,eva_nr,station,category,city,count
0,8000001,Aachen Hbf,2,Aachen,1488
1,8000002,Aalen Hbf,3,Aalen,823
2,8000004,Altenbeken,3,Altenbeken,848
3,8000007,Alzey,3,Alzey,591
4,8000009,Ansbach,3,Ansbach,829
...,...,...,...,...,...
1991,8089472,Lichterfelde Süd,5,Berlin,1544
1992,8089473,Osdorferstraße,5,Berlin,1536
1993,8089474,Schöneberg,4,Berlin,5831
1994,8089537,Julius-Leber-Brücke,5,Berlin,2191


In [9]:
station_mapping_df.count()

eva_nr      1996
station     1996
category    1996
city        1996
count       1996
dtype: int64

what is the average number of stations per category? go check the 'category' column

In [10]:
# Step 1: Split the 'path' into individual stations where 'path' is not empty
# Create a new DataFrame that splits the 'path' column
stations_df = train_rides_df['path'].str.split('|', expand=True)

# Step 2: Keep the same 'eva_nr', 'station', and 'city' columns, add the split station columns
split_stations_df = pd.concat([train_rides_df[['eva_nr', 'station', 'city']], stations_df], axis=1)

split_stations_df

,eva_nr,station,city,0,1,2,3,4,5,6,...,49,50,51,52,53,54,55,56,57,58
0,8000001,Aachen Hbf,Aachen,Stolberg(Rheinl)Hbf Gl.44,Eschweiler-St.Jöris,Alsdorf Poststraße,Alsdorf-Mariadorf,Alsdorf-Kellersberg,Alsdorf-Annapark,Alsdorf-Busch,...,None,None,None,None,None,None,None,None,None,None
1,8000001,Aachen Hbf,Aachen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None
2,8000406,Aachen-Rothe Erde,Aachen,Hamm(Westf)Hbf,Kamen,Kamen-Methler,Dortmund-Kurl,Dortmund-Scharnhorst,Dortmund Hbf,Bochum Hbf,...,None,None,None,None,None,None,None,None,None,None
3,8000404,Aachen West,Aachen,Aachen Hbf,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,8000404,Aachen West,Aachen,Herzogenrath,Kohlscheid,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2061352,8002795,Herten (Westf),Herten,Bottrop Hbf,Bottrop-Boy,Gladbeck West,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2061353,8003775,Lübeck-Moisling,Lübeck,Lübeck-Travemünde Strand,Lübeck-Travemünde Hafen,Lübeck-Travem. Skandinavienkai,Lübeck-Kücknitz,Lübeck-Dänischburg IKEA,Lübeck Hbf,None,...,None,None,None,None,None,None,None,None,None,None
2061354,8003775,Lübeck-Moisling,Lübeck,Bad Oldesloe,Reinfeld(Holst),None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2061355,8001580,Düsseldorf Flughafen Terminal,Düsseldorf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,None


In [11]:
split_stations_df.head()

,eva_nr,station,city,0,1,2,3,4,5,6,...,49,50,51,52,53,54,55,56,57,58
0,8000001,Aachen Hbf,Aachen,Stolberg(Rheinl)Hbf Gl.44,Eschweiler-St.Jöris,Alsdorf Poststraße,Alsdorf-Mariadorf,Alsdorf-Kellersberg,Alsdorf-Annapark,Alsdorf-Busch,...,None,None,None,None,None,None,None,None,None,None
1,8000001,Aachen Hbf,Aachen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,8000406,Aachen-Rothe Erde,Aachen,Hamm(Westf)Hbf,Kamen,Kamen-Methler,Dortmund-Kurl,Dortmund-Scharnhorst,Dortmund Hbf,Bochum Hbf,...,None,None,None,None,None,None,None,None,None,None
3,8000404,Aachen West,Aachen,Aachen Hbf,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,8000404,Aachen West,Aachen,Herzogenrath,Kohlscheid,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


Visualise the stations in a map (/grid) (without it being mapped in a real map) and see if there are any patterns. George, Additionally, uses the same map to see the delay (in % of total rides) per station. George
See if we can use K-means to cluster the stations into mini regions to see if the entire region has delay problems. We do not know what k to use, so we can try a few values. Check the distance per stations and see by eye if the clusters make sense to create. Raphael